This notebook can be used to automatically generate samples from valid images and add them to a custom vision model with training tags applied. To please go to <a href='https://www.customvision.ai/'>Custom Vision Portal</a> to retrieve keys and verify folders for input and output of samples.

In [8]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry
import os
from PIL import Image
from math import tan
import numpy as np
import random
import cv2
import math

ENDPOINT = "https://XXX.cognitiveservices.azure.com/"

# Replace with a valid keys
training_key = "XXX"
prediction_key = "XXX"
prediction_resource_id = "XXX"
projectid = 'XXX'
numOfSamples = 10
generateFlipped = True
inputImgDir = './Input/'
sampleImgDir = "./Output/"
processedImgDir = "./Processed/"

trainer = CustomVisionTrainingClient(training_key, endpoint=ENDPOINT)
project = trainer.get_project(projectid)

tags = trainer.get_tags(projectid)


In [5]:
from os import listdir
from os.path import isfile, join
path = './Input'
charfiles = [f for f in listdir(path) if isfile(join(path, f))]
charset = set()

for char in charfiles:
    #print(char.split('.')[0])
    charset.add(char.split('.')[0])

charset

{'10c',
 '10d',
 '10h',
 '10s',
 '9c',
 '9d',
 '9h',
 '9s',
 'Kc',
 'Kh',
 'Ks',
 'Qc',
 'Qd',
 'Qh',
 'Qs'}

In [6]:
for char in charset:
    print(char)
    trainer.create_tag(projectid, char)

10c
9s
10h
10d
9c
9d
10s
Qs
Qc
Qd
Kh
Kc
9h
Qh
Ks


In [9]:
tags = trainer.get_tags(projectid)
tagdict = {}
for tag in tags:
    tagdict[tag.name] = tag.id
    
tagdict

{'9s': '7804fbad-9636-4e1d-889b-0fe9311b8336',
 '10c': 'e2472c08-12e6-4719-a44a-1a878119f8b6',
 'Qc': '39d6f009-a1da-4765-a46f-2675b1c4be4f',
 '9c': '01c2ae69-9a19-4d8e-97e7-33ccc901a6d5',
 '9h': 'ebc5e01c-5213-477d-a856-412088f45757',
 '10s': '842db521-c42a-400d-8be1-4a0ee27e704f',
 'Qh': '0278c0d9-25cb-4254-b431-64859e140d97',
 'Ks': 'd2e2828d-43d4-4fb3-9d0b-9d3a38c702b9',
 '9d': '078774b0-ee0e-4e87-a950-9e07659e954e',
 'Qd': 'ea442304-241e-4e09-83e9-9e75f8b8f239',
 '10d': '2756568a-60ba-47ef-9bcb-a19202d16919',
 'Qs': 'aa5f985f-3fa5-4655-a526-b431dad29e14',
 '10h': 'be28551a-45d5-4bd2-96ff-c260a746c99f',
 'Kh': 'c1ac82b5-dc03-4dfd-b8ca-cfadf8779ddb',
 'Kc': '213cc0a4-8c3e-4508-b0ba-fe2d08357cb9'}

In [22]:
image_list = []

directory = os.fsencode(sampleImgDir)
batch = 0
# Iterate over each file in Input Folder
for file in os.listdir(directory):
    fname = os.fsdecode(file)
    with open("{}{}".format(sampleImgDir,fname), "rb") as image_contents:
        tagnm = fname.split('.')[0]
        tagid = tagdict[tagnm]
        #print(fname, tagid)
        image_list.append(ImageFileCreateEntry(name=fname, contents=image_contents.read(), tag_ids=[tagid]))
        batch = batch+1
    if(batch >= 50) :
        print("Adding {} images to training set...".format(len(image_list)))
        upload_result = trainer.create_images_from_files(project.id, images=image_list)
        if not upload_result.is_batch_successful:
            print("Image batch upload failed.")
            for image in upload_result.images:
                print("Image status: ", image.status)
            exit(-1)
        else :
            print("Image batch success!")
        image_list = []
        batch=0

if (len(image_list) > 0) :
    print("Adding {} images to training set...".format(len(image_list)))
    upload_result = trainer.create_images_from_files(project.id, images=image_list)
    if not upload_result.is_batch_successful:
        print("Image batch upload failed.")
        for image in upload_result.images:
            print("Image status: ", image.status)
        exit(-1)
    else :
        print("Image batch success!")




Adding 15 images to training set...
Image batch upload failed.
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
